<a href="https://colab.research.google.com/github/Rhitabrat/deep-learning-CS510/blob/main/hw3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW 3

In [84]:
import tensorflow as tf 
from tensorflow.keras.applications import ResNet50
from tensorflow import keras
import numpy as np
import pandas as pd

In [85]:
(X_train_ori, y_train), (X_test_ori, y_test) = keras.datasets.mnist.load_data()
assert X_train_ori.shape == (60000, 28, 28)
assert X_test_ori.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [86]:
def select_data(n, X_train, y_train):
    classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    indices_all_classes = []

    for each_class in classes:
        indices = [i for i, x in enumerate(y_train) if x == each_class]
        indices_all_classes.append(indices)

    new_X_train = []
    new_y_train = []

    for class_indices in indices_all_classes:
        x, y = X_train[class_indices[:n]], y_train[class_indices[:n]]
        new_X_train.extend(x)
        new_y_train.extend(y)

    return new_X_train, new_y_train

##Exp #1

In [87]:
def prepare_data(X_train, y_train):
    X_train = np.expand_dims(X_train, axis=-1)
    X_train = np.repeat(X_train, 3, axis=-1)
    X_train = X_train.astype('float32') / 255
    X_train = tf.image.resize(X_train, [224, 224])
    y_train = tf.keras.utils.to_categorical(y_train , num_classes=10)
    return X_train, y_train

In [88]:
def shuffle_data(X_train, y_train, N):
    indices = tf.range(start=0, limit=10*N, dtype=tf.int32)
    shuffled_indices = tf.random.shuffle(indices)

    shuffled_x = tf.gather(X_train, shuffled_indices)
    shuffled_y = tf.gather(y_train, shuffled_indices)
    return shuffled_x, shuffled_y

In [89]:
# 10x10 data - 10 from each category

N=10
X_train, y_train = select_data(N, X_train_ori, y_train)
X_train, y_train = prepare_data(X_train, y_train)
X_train, y_train = shuffle_data(X_train, y_train, N)

In [96]:
X_test, y_test = prepare_data(X_test, y_test)

In [90]:
resnet_model = ResNet50(weights="imagenet", 
                        include_top=False)

resnet_model.trainable = False

In [91]:
# inputs = keras.Input(shape=(224, 224, 3))

# # Separately from setting trainable on the model, we set training to False 
# x = model(inputs, training=False)
# x = keras.layers.GlobalAveragePooling2D()(x)
# # x = keras.layers.Flatten()(x)

# outputs = keras.layers.Dense(10, activation="relu")(x)

# model = keras.Model(inputs, outputs)


model = keras.models.Sequential()
model.add(keras.Input(shape=(224, 224, 3)))
model.add(resnet_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))

In [92]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 flatten_4 (Flatten)         (None, 100352)            0         
                                                                 
 dense_7 (Dense)             (None, 10)                1003530   
                                                                 
Total params: 24,591,242
Trainable params: 1,003,530
Non-trainable params: 23,587,712
_________________________________________________________________


In [93]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.CategoricalCrossentropy(name="categorial_loss"),
              metrics=["accuracy"])

In [98]:
model.fit(X_train, y_train, 
          steps_per_epoch=8, 
          validation_split=0.2,
          epochs=10)

Epoch 1/10
8/8 [==============================] - 2s 60ms/step - loss: 0.7386 - accuracy: 0.8125 - val_loss: 0.3098 - val_accuracy: 0.9500
Epoch 2/10
8/8 [==============================] - 0s 36ms/step - loss: 0.6273 - accuracy: 0.7750 - val_loss: 0.9788 - val_accuracy: 0.6500
Epoch 3/10
8/8 [==============================] - 0s 36ms/step - loss: 0.9721 - accuracy: 0.8125 - val_loss: 0.5491 - val_accuracy: 0.8500
Epoch 4/10
8/8 [==============================] - 0s 35ms/step - loss: 0.7245 - accuracy: 0.7375 - val_loss: 0.6644 - val_accuracy: 0.8500
Epoch 5/10
8/8 [==============================] - 0s 36ms/step - loss: 1.0404 - accuracy: 0.7500 - val_loss: 1.8444 - val_accuracy: 0.5500
Epoch 6/10
8/8 [==============================] - 0s 35ms/step - loss: 1.2042 - accuracy: 0.6750 - val_loss: 1.2710 - val_accuracy: 0.6500
Epoch 7/10
8/8 [==============================] - 0s 35ms/step - loss: 1.1688 - accuracy: 0.6750 - val_loss: 1.3008 - val_accuracy: 0.7000
Epoch 8/10
8/8 [===========

In [97]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 20s 61ms/step - loss: 1.4343 - accuracy: 0.5531


[1.4343338012695312, 0.5530999898910522]